# Spark DataFrames Project Exercise 

Let's get some quick practice with your new Spark DataFrame skills, you will be asked some basic questions about some stock market data, in this case Walmart Stock from the years 2012-2017. This exercise will just ask a bunch of questions, unlike the future machine learning exercises, which will be a little looser and be in the form of "Consulting Projects", but more on that later!

For now, just answer the questions and complete the tasks below.

#### Use the walmart_stock.csv file to Answer and complete the  tasks below!

#### Start a simple Spark Session

In [1]:
from pyspark.sql import SparkSession


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1666734571482_0005,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
spark = SparkSession.builder.appName('SparkHDInsightExample').getOrCreate()

In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

#### Load the Walmart Stock CSV File, have Spark infer the data types.

In [113]:
df = spark.read.options(inferSchema='True',header=True).csv("wasb:///HdiNotebooks/walmart_stock.csv")

#### What are the column names?

In [11]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### What does the Schema look like?

In [12]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)

#### Print out the first 5 columns.

In [114]:
df.tail(5)

[Row(Date='2016-12-23', Open=69.43, High=69.75, Low=69.360001, Close=69.540001, Volume=4803900, Adj Close=69.032411), Row(Date='2016-12-27', Open=69.300003, High=69.82, Low=69.25, Close=69.699997, Volume=4435700, Adj Close=69.19124000000001), Row(Date='2016-12-28', Open=69.940002, High=70.0, Low=69.260002, Close=69.309998, Volume=4875700, Adj Close=68.804087), Row(Date='2016-12-29', Open=69.209999, High=69.519997, Low=69.120003, Close=69.260002, Volume=4298400, Adj Close=68.754456), Row(Date='2016-12-30', Open=69.120003, High=69.43, Low=68.83000200000001, Close=69.120003, Volume=6889500, Adj Close=68.61547900000001)]

#### Use describe() to learn about the DataFrame.

In [25]:
df.describe().show()

+-------+----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|      Date|             Open|             High|              Low|            Close|           Volume|         AdjClose|
+-------+----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|      1258|             1258|             1258|             1258|             1258|             1258|             1258|
|   mean|      null|72.35785373608894|72.83938791732918|71.91860095389515|72.38844992050863|8222093.481717011|67.23871224165353|
| stddev|      null| 6.76809037021648|6.768186744823553|6.744075695823193|6.756859155425468|  4519780.8431556|6.722615952565601|
|    min|2012-01-03|            56.39|            57.06|             56.3|            56.42|        2094900.0|            50.36|
|    max|2016-12-30|             90.8|            90.97|            89.25|            90.47|     

## Bonus Question!
#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. [Check this link for a hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast)

If you get stuck on this, don't worry, just view the solutions.

In [119]:
df1 = df.select(df.Open, df.High, df.Low,df.Close,df.Volume).describe()

In [120]:
df1 = df1.withColumn('Open', round(df1.Open.cast(DoubleType()),2))
df1 = df1.withColumn('High', round(df1.High.cast(DoubleType()),2))
df1 = df1.withColumn('Low', round(df1.Low.cast(DoubleType()),2))
df1 = df1.withColumn('Close', round(df1.Close.cast(DoubleType()),2))
df1 = df1.withColumn('Volume', round(df1.Volume.cast(DoubleType()),2))

In [121]:
df1.show()

+-------+------+------+------+------+----------+
|summary|  Open|  High|   Low| Close|    Volume|
+-------+------+------+------+------+----------+
|  count|1258.0|1258.0|1258.0|1258.0|    1258.0|
|   mean| 72.36| 72.84| 71.92| 72.39|8222093.48|
| stddev|  6.77|  6.77|  6.74|  6.76|4519780.84|
|    min| 56.39| 57.06|  56.3| 56.42| 2094900.0|
|    max|  90.8| 90.97| 89.25| 90.47| 8.08981E7|
+-------+------+------+------+------+----------+

#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [39]:
df2 = df.select((df.High/df.Volume).alias('HV Ratio'))
df2.show()

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714574387472E-6|
|6.290848821573389...|
|4.669413073103491E-6|
|7.367338339901356E-6|
|8.915604928660188E-6|
|8.644477581688938E-6|
| 9.35182857861003E-6|
| 8.29141562102703E-6|
|7.712211972623653E-6|
|7.071764705882352...|
|1.015495483303447...|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448324825044...|
|8.316075414862431E-6|
|9.721183974042911E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows

#### What day had the Peak High in Price?

In [107]:
val1 = df.select(max('High').alias('High'))
df.join(val1,'High').select(df.Date).show()

+----------+
|      Date|
+----------+
|2015-01-13|
+----------+

#### What is the mean of the Close column?

In [45]:
df.select(mean('Close')).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844992050863|
+-----------------+

#### What is the max and min of the Volume column?

In [46]:
df.select(max('Volume'),min('Volume')).show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|  8.08981E7|  2094900.0|
+-----------+-----------+

#### How many days was the Close lower than 60 dollars?

In [96]:
df3 = df.filter(df.Close.cast(DoubleType()) < 60).select(count('Date').alias('Number of Days'))
df3.show()

+--------------+
|Number of Days|
+--------------+
|            81|
+--------------+

#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [97]:
dfHigh80 = df.filter(df.High.cast(DoubleType()) > 80).select(count('Date').alias('Days80'))
dfTotal = df.select(count('Date').alias('Total'))
df4 = dfHigh80.crossJoin(dfTotal)
df4.select((col('Days80')/col('Total')*100).alias('Percentage')).show()

+-----------------+
|       Percentage|
+-----------------+
|9.141494435612083|
+-----------------+

#### What is the Pearson correlation between High and Volume?
#### [Hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameStatFunctions.corr)

In [74]:
df.select(corr('High','Volume')).show()

+--------------------+
|  corr(High, Volume)|
+--------------------+
|-0.33843260582148915|
+--------------------+

#### What is the max High per year?

In [98]:
df5 = df.withColumn('Year',year(to_date(col("Date"),"yyyy-mm-dd"))).groupBy('Year').max('High').sort('Year')
df5.show()

+----+---------+
|Year|max(High)|
+----+---------+
|2012|77.599998|
|2013|81.370003|
|2014|88.089996|
|2015|90.970001|
|2016|75.190002|
+----+---------+

#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months. 

In [124]:
df6 = df.withColumn('Date',to_date(col("Date"),"yyyy-mm-dd"))
df6 = df.withColumn('Month',date_format('Date','M'))
df7 = df6.groupBy('Month').avg('Close').sort(df6.Month.cast(IntegerType()))
df7.show()

+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|    1|71.44801958415842|
|    2|  71.306804443299|
|    3|71.77794377570092|
|    4|72.97361900952382|
|    5|72.30971688679247|
|    6| 72.4953774245283|
|    7|74.43971943925233|
|    8|73.02981855454546|
|    9|72.18411785294116|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|   12|72.84792478301885|
+-----+-----------------+

# Great Job!